In [234]:
# basic functionalities
import os
import sys
import datetime
import itertools
import math

# data transforamtion and manipulation
import pandas as pd
import numpy as np
from math import sqrt
import ydata_profiling

# prevent crazy long numpy prints
np.set_printoptions(precision=4, suppress=True)

# prevent crazy long pandas prints
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)


# remove warnings
import warnings
warnings.filterwarnings('ignore')


# plotting and plot styling
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go


# set params
plt.rcParams['figure.figsize'] = (16,10)
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['text.color'] = 'k'
plt.style.use('fivethirtyeight')

# statistical modeling libraries
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.arima_model import ARMA, ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX


# ML basic library
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error, mean_squared_log_error

#Img:
from IPython.display import Image

In [235]:
df = pd.read_csv("./anonimized/25day_dataset.csv")

df['ActivePower'] /= 1000
df['ReactivePower'] /= 1000
df['wahing_machine'] /= 1000
df['dishwasher'] /= 1000
df['oven'] /= 1000

In [236]:

df['DateTime'] = pd.to_datetime(df['DateTime'])

df = df.sort_values(by=['DateTime'], axis=0, ascending=True)


In [237]:
df.drop_duplicates(subset='DateTime', keep='last', inplace=True)
df = df.set_index('DateTime')

In [238]:
date_range = pd.date_range(start=min(df.index), end=max(df.index), freq='S')

In [239]:
df = df.reindex(date_range)

for col in df.columns:
    df[col].interpolate(method='linear', inplace=True)

In [240]:
df = df.resample("1T").mean()

In [241]:
df['dayofweek'] = df.index.dayofweek
df['dayofyear'] = df.index.dayofyear
df['year'] = df.index.year
df['month'] = df.index.month
df['quarter'] = df.index.quarter
df['hour'] = df.index.hour
df['weekday'] = df.index.weekday
df['weekofyear'] = df.index.dayofyear//7
df['dayofmonth'] = df.index.day # Day of Month
df['date'] = df.index.date 

# let's add the season number
df['season'] = df['month'].apply(lambda month_number: (month_number%12 + 3)//3)

In [242]:
y_wm = (df['wahing_machine']>0).astype(int)
y_dw = (df['dishwasher']>0).astype(int)
y_ow = (df['oven']>0).astype(int)

drop_cols = ['wahing_machine', 'dishwasher', 'oven', 'date']
X =  df.drop(drop_cols, axis=1)
Y = pd.concat([y_wm, y_dw, y_ow], axis=1)


Y['label'] = Y[['wahing_machine','dishwasher','oven']].apply(lambda row: int(''.join(map(str, row)), 2), axis=1)
Y['label_str'] = Y['label'].map({0:'AO',
                                1:'OW',
                                2:'DW',
                                3:'DW_OW',
                                4:'WM',
                                5:'WM_OW',
                                6:'WM_DW',
                                7:'WM_DW_OW',
                                })

y = Y['label_str']

In [243]:
target_names = ['AO','OW','DW','DW_OW','WM','WM_OW','WM_DW','WM_DW_OW']

In [244]:
def sencos(df, col):
	cos_name = "cos_" + col
	sen_name = "sen_" + col
	df[cos_name] = np.cos(2 * np.pi * df[col] / df[col].max())
	df[sen_name] = np.sin(2 * np.pi * df[col] / df[col].max())
	df.drop(col, axis=1, inplace=True)
	return df

df.drop(['month', 'quarter', 'year', 'dayofyear', 'weekofyear', 'season', 'dayofweek', 'dayofmonth'], inplace=True, axis=1)

In [245]:
drop_cols = ['wahing_machine', 'dishwasher', 'oven']

In [246]:
df = sencos(df, 'hour')
df = sencos(df, 'weekday')

In [247]:
df.drop(['date'], axis=1, inplace=True)

apparent = np.sqrt(df['ActivePower']**2 + df['ReactivePower']**2)
#df['ApparentPower'] = apparent
df['PowerFactor'] = df['ActivePower'] / apparent
#df['Impedance'] = apparent / df['ActivePower']
#df['Resistance'] = df['ActivePower'] / apparent
#df['Reactance'] = np.sqrt(df['Impedance']**2 - df['Resistance']**2)
#df['Admittance'] = 1 / df['Impedance']
#df['Conductance'] = 1 / df['Resistance']
#df['Susceptance'] = 1 / df['Reactance']
#df['RealPower'] = df['ActivePower'] * df['PowerFactor']

df = df.dropna()

In [248]:
from scipy.signal import find_peaks, peak_widths, peak_prominences, cwt
from scipy.signal import ricker
from scipy import signal

widths = np.arange(1, 20)
cwtmatr = cwt(df['ActivePower'], signal.ricker, widths)
cw = pd.DataFrame(cwtmatr.T, columns=widths)
cw.index = df.index
cw.columns = ['cwt_' + str(col) for col in cw.columns]

#df = pd.concat([df, cw], axis=1)

In [249]:
df.columns

Index(['ActivePower', 'ReactivePower', 'Voltage', 'Current', 'harmonic1_Real', 'harmonic1_Imaginary', 'harmonic3_Real', 'harmonic3_Imaginary', 'harmonic5_Real', 'harmonic5_Imaginary', 'harmonic7_Real', 'harmonic7_Imaginary', 'wahing_machine', 'dishwasher', 'oven', 'cos_hour', 'sen_hour', 'cos_weekday', 'sen_weekday', 'PowerFactor'], dtype='object')

In [250]:
df = df.dropna()
#df = df.drop(['Voltage'], axis=1)
df.columns

Index(['ActivePower', 'ReactivePower', 'Voltage', 'Current', 'harmonic1_Real', 'harmonic1_Imaginary', 'harmonic3_Real', 'harmonic3_Imaginary', 'harmonic5_Real', 'harmonic5_Imaginary', 'harmonic7_Real', 'harmonic7_Imaginary', 'wahing_machine', 'dishwasher', 'oven', 'cos_hour', 'sen_hour', 'cos_weekday', 'sen_weekday', 'PowerFactor'], dtype='object')

In [251]:
for i in range(1, 9):
	df['ActivePower_shifted_' + str(i)] = df['ActivePower'].shift(i)

df['ActivePower'] = df['ActivePower'] - df['ActivePower'].rolling(window=10).mean()

#for i in range(1, 21):
	#df['cwt_9_shifted_' + str(i)] = cw['cwt_9'].shift(i)

#df['cwt_1'] = cw['cwt_1']
#df['cwt_2'] = cw['cwt_2']
#df['cwt_3'] = cw['cwt_3']
#df['cwt_4'] = cw['cwt_4']
#df['cwt_5'] = cw['cwt_5']
#df['cwt_6'] = cw['cwt_6']
#df['cwt_7'] = cw['cwt_7']
#df['cwt_8'] = cw['cwt_8']
#df['cwt_9'] = cw['cwt_9']


#droppo nan
df = df.dropna()

In [252]:
signale = df['ActivePower']
fourier = np.fft.fft(df['ActivePower'])


#estraggo la parte reale e immaginaria e le aggiungo al dataset

df['fourier_real'] = fourier.real
df['fourier_imag'] = fourier.imag

In [253]:
df.columns

Index(['ActivePower', 'ReactivePower', 'Voltage', 'Current', 'harmonic1_Real', 'harmonic1_Imaginary', 'harmonic3_Real', 'harmonic3_Imaginary', 'harmonic5_Real', 'harmonic5_Imaginary', 'harmonic7_Real', 'harmonic7_Imaginary', 'wahing_machine', 'dishwasher', 'oven', 'cos_hour', 'sen_hour', 'cos_weekday', 'sen_weekday', 'PowerFactor', 'ActivePower_shifted_1', 'ActivePower_shifted_2', 'ActivePower_shifted_3', 'ActivePower_shifted_4', 'ActivePower_shifted_5', 'ActivePower_shifted_6', 'ActivePower_shifted_7', 'ActivePower_shifted_8', 'fourier_real', 'fourier_imag'], dtype='object')

In [254]:
#seleziono whahing machine == 0 e oven == 0 e dishwasher == 0 e creo un nuovo dataset

dfoff = df[(df['wahing_machine'] == 0) & (df['oven'] == 0) & (df['dishwasher'] == 0)]	

#seleziono whahing machine == 1 e oven == 0 e dishwasher == 0 e creo un nuovo dataset

dfwm = df[(df['wahing_machine'] != 0) & (df['oven'] == 0) & (df['dishwasher'] == 0)]

#seleziono whahing machine == 0 e oven == 1 e dishwasher == 0 e creo un nuovo dataset

dfov = df[(df['wahing_machine'] == 0) & (df['oven'] != 0) & (df['dishwasher'] == 0)]

#seleziono whahing machine == 0 e oven == 0 e dishwasher == 1 e creo un nuovo dataset

dfdw = df[(df['wahing_machine'] == 0) & (df['oven'] == 0) & (df['dishwasher'] != 0)]

In [255]:
frequenze = np.fft.fftfreq(len(df['ActivePower']))
len(frequenze)

df['frequenze'] = frequenze

In [256]:
y_wm = (df['wahing_machine']>0).astype(int)
y_dw = (df['dishwasher']>0).astype(int)
y_ow = (df['oven']>0).astype(int)

Y = pd.concat([y_wm, y_dw, y_ow], axis=1)


Y['label'] = Y[['wahing_machine','dishwasher','oven']].apply(lambda row: int(''.join(map(str, row)), 2), axis=1)
Y['label_str'] = Y['label'].map({0:'AO',
                                1:'OW',
                                2:'DW',
                                3:'DW_OW',
                                4:'WM',
                                5:'WM_OW',
                                6:'WM_DW',
                                7:'WM_DW_OW',
                                })

y = Y['label_str']

In [257]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.multiclass import OneVsRestClassifier

X =  df.drop(drop_cols, axis=1)

clf = OneVsRestClassifier(SVC(kernel='rbf', probability=True, random_state=42, class_weight='balanced', C=1))
scaler = StandardScaler()
X = scaler.fit_transform(X)

#X = SelectKBest(f_classif, k=25).fit_transform(X, y)

tscv = TimeSeriesSplit(n_splits=5)
for train_index, test_index in tscv.split(X):
	X_train, X_test = X[train_index], X[test_index]
	y_train, y_test = y.iloc[train_index], y.iloc[test_index]
	clf.fit(X_train, y_train)
	print(classification_report(y_test, clf.predict(X_test)))


              precision    recall  f1-score   support

          AO       0.98      0.99      0.99      5535
          DW       0.88      0.63      0.74        73
          OW       0.40      0.34      0.37        53
          WM       0.93      0.80      0.86       337

    accuracy                           0.97      5998
   macro avg       0.80      0.69      0.74      5998
weighted avg       0.97      0.97      0.97      5998

              precision    recall  f1-score   support

          AO       0.98      0.98      0.98      5465
          DW       0.58      0.56      0.57        71
          OW       0.64      0.67      0.65        55
          WM       0.82      0.87      0.84       407

    accuracy                           0.96      5998
   macro avg       0.75      0.77      0.76      5998
weighted avg       0.96      0.96      0.96      5998

              precision    recall  f1-score   support

          AO       0.99      0.90      0.94      5306
          DW       0.

In [258]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=0)


clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          AO       0.98      0.99      0.99     29378
          DW       0.82      0.79      0.80       488
          OW       0.87      0.61      0.72       297
          WM       0.85      0.87      0.86      2228
       WM_DW       0.00      0.00      0.00         1

    accuracy                           0.97     32392
   macro avg       0.70      0.65      0.67     32392
weighted avg       0.97      0.97      0.97     32392

